# Accessibility Analysis using OpenRouteService

In [1]:
import folium
from folium import Map, Marker, FeatureGroup, LayerControl

from openrouteservice import client, geocode, directions, places

import numpy as np
from matplotlib import pylab

import fiona as fn
import geojson 

from pprint import pprint

In [2]:
api_key = '58d904a497c67e00015b45fc2a6b6872037d44119582ef40cdf264c4' # Provide your personal API key
clnt = client.Client(key=api_key)
map_madagascar = folium.Map(tiles='Stamen Toner', location=([-18.812718, 46.713867]), zoom_start=5)

# Shapefiles from https://data.humdata.org/dataset/madagascar-administrative-boundary-shapefiles-level-1-4 (05/07/2018)
district_data = []
with fn.open('mdg_polbnda_adm2_Distritcts_BNGRC_OCHA.shp', 'r') as districts:
    for polygon in districts:
        folium.GeoJson(polygon).add_to(map_madagascar)
        district_data.append(polygon)
    
# import geojson as gj
# Health facilities from https://data.humdata.org/dataset/madagascar-healthsites (05/07/2018)
with open('health_facilities.geojson') as facility_data:
    facility = geojson.load(facility_data)
    folium.GeoJson(facility).add_to(map_madagascar)
    
#map_madagascar

In [3]:
print('There are {} health facilities.'.format(len(facility['features'])))

There are 121 health facilities.


In [8]:
def style_function(color):
    return lambda feature: dict(color=color)

# 1 hour Isochrones for foot walking and car driving
iso_foot = []
for loc in facility['features']:
    iso_params = {'locations': loc['geometry']['coordinates'],
                  'profile': 'foot-walking',
                  'range_type': 'time',
                  'segments': 3600, # 1 hour
                  'attributes': {'total_pop', 'area'}}
    iso_foot_request = clnt.isochrones(**iso_params)
    #folium.GeoJson(iso, style_function=style_function('#ace600')).add_to(map_madagascar)
    iso_foot.append(iso_foot_request)

Request limit of 40 per minute exceeded. Wait for 59.27603030204773 seconds
Request limit of 40 per minute exceeded. Wait for 0.0006494522094726562 seconds
Request limit of 40 per minute exceeded. Wait for 0.009759187698364258 seconds
Request limit of 40 per minute exceeded. Wait for 0.0042552947998046875 seconds
Request limit of 40 per minute exceeded. Wait for 0.014998435974121094 seconds


ApiError: 500 ({'error': {'code': 3099, 'message': 'Unable to build an isochrone map.'}, 'info': {'engine': {'version': '4.5.0', 'build_date': '2018-06-12T17:36:31Z'}, 'timestamp': 1531206109813}})

In [22]:
iso_car = []
for loc in facility['features']:
    del iso_params['profile']
    iso_params['profile'] = 'driving-car'
    iso_car_request = clnt.isochrones(**iso_params)
    #folium.GeoJson(iso, style_function=style_function('#ffd699')).add_to(map_madagascar)
    iso_car.append(iso_car_request)

Request limit of 40 per minute exceeded. Wait for 54.51008582115173 seconds
Request limit of 40 per minute exceeded. Wait for 0.2702901363372803 seconds
Request limit of 40 per minute exceeded. Wait for 0.2873845100402832 seconds
Request limit of 40 per minute exceeded. Wait for 0.11726069450378418 seconds
Request limit of 40 per minute exceeded. Wait for 0.10579466819763184 seconds
Request limit of 40 per minute exceeded. Wait for 0.07680439949035645 seconds
Request limit of 40 per minute exceeded. Wait for 0.1635444164276123 seconds


ApiError: 500 ({'error': {'code': 3099, 'message': 'Unable to build an isochrone map.'}, 'info': {'engine': {'version': '4.5.0', 'build_date': '2018-06-12T14:47:57Z'}, 'timestamp': 1530810660806}})

In [25]:
import pandas as pd
# population in isochrones -> access to pois
plot_data = []
for iso in iso_foot:
    coordinates = iso['features'][0]['properties']['center']
    #area = iso['features'][0]['properties']['area']
    pop = iso['features'][0]['properties']['total_pop']
    total_area_km = iso['features'][0]['properties']['total_area_km']
    plot_data += [[coordinates, pop, total_area_km]]
    
df = pd.DataFrame(data=plot_data, columns = ['Location', 'Total Population', 'Total Area'])
display(df)
    

,Location,Total Population,Total Area
0,"[47.531007, -18.908284]",579875,52.6763
1,"[47.531007, -18.908284]",579875,52.6763
2,"[47.531007, -18.908284]",579875,52.6763
3,"[47.531007, -18.908284]",579875,52.6763
4,"[47.531007, -18.908284]",579875,52.6763
5,"[47.531007, -18.908284]",579875,52.6763
6,"[47.531007, -18.908284]",579875,52.6763
7,"[47.531007, -18.908284]",579875,52.6763
8,"[47.531007, -18.908284]",579875,52.6763
9,"[47.531007, -18.908284]",579875,52.6763


In [49]:
# Polygon geometry is too large 

#POI
#poi_health = []
#for geojson in district_geojson:
 #   poi_params = {'request': 'pois',
  #              'geojson': geojson['geometry'],
   #             'filter_category_ids': [202, 204, 205, 206, 208]} # 202=clinic, 204=doctors, 205=emergency_access_point, 206=hospital, 208=pharmacy
   # poi_request = clnt.places(**poi_params)
    #poi_health.append(poi_request)

ApiError: 500 ({'code': 4008, 'message': 'Your polygon geometry is too large (653562597.888687 square meters), check the server restrictions.'})